In [2]:
import zipfile
import pandas as pd

## Unzip Cluster Dataset

In [3]:
# Percorso del file ZIP da estrarre
zip_file_path = "real-time clusters/20230101-20240101_real_time_clusters.parquet.zip"

# Percorso della cartella di destinazione per l'estrazione
extract_to_folder = "real-time clusters"

# Aprire il file ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Estrarre tutto il contenuto nella cartella di destinazione
    zip_ref.extractall(extract_to_folder)

In [3]:
real_time_clusters_df = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [4]:
real_time_clusters_df

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13
39,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9
40,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13
41,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13


In [5]:
real_time_clusters_df.columns

Index(['alarm_id', 'network', 'ne_type', 'first_occurrence', 'last_update',
       'data', 'ne_id', 'geo_code', 'min_cluster_first_occurrence', 'domain',
       'link_cluster', 'cluster_id', 'cluster_id2'],
      dtype='object')

In [6]:
clusters = real_time_clusters_df.groupby(["cluster_id2"])

In [7]:
num_of_clusters = len(clusters)

In [8]:
num_of_clusters

3451405

In [9]:
clusters.size().sort_values(ascending=False)

cluster_id2
202309181126_19    10079
202308080334_40     7989
202311270204_5      6728
202311270406_7      6611
202310110506_13     6226
                   ...  
202307101348_20        1
202307101348_21        1
202307101348_22        1
202307101348_3         1
202312312358_9         1
Length: 3451405, dtype: int64

In [10]:
# Supponiamo che tu abbia un DataFrame raggruppato groupby_df
clusters_filtered = clusters.filter(lambda x: len(x) >= 3)

In [11]:
clusters_filtered

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
12,COS1P;136899249,sdh,ad64,2022-12-31 18:16:45,2022-12-31 23:59:58,2023-01-01 00:00:02,beneital/ad64/2,bene,2022-12-31 18:16:45,tx,None,202301010000_27,202301010000_12
16,COS1P;136885076,sdh,ad16,2022-12-31 03:18:22,2022-12-31 23:59:59,2023-01-01 00:00:02,bracitah/ad16/1,brac,2022-12-31 03:18:22,tx,None,202301010000_31,202301010000_16
17,COS1P;136885115,sdh,ad16,2022-12-31 03:18:26,2022-12-31 23:59:50,2023-01-01 00:00:02,bracitam/ad16/3,brac,2022-12-31 03:18:26,tx,None,202301010000_32,202301010000_16
26,COS1P;136884642,sdh,adm4,2022-12-31 03:15:35,2022-12-31 23:59:52,2023-01-01 00:00:02,capoitam/adm4/1,capo,2022-12-31 03:15:35,tx,None,202301010000_37,202301010000_25
27,COS1P;136884882,sdh,ad16,2022-12-31 03:17:37,2022-12-31 23:59:56,2023-01-01 00:00:02,capoitaq/ad16/1,capo,2022-12-31 03:17:37,tx,None,202301010000_38,202301010000_25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,EFM;6591ef405795e30001315adb,an,lebts,2023-12-31 23:46:24,2023-12-31 23:56:44,2023-12-31 23:58:03,1btsrm155,romm,2023-12-31 23:46:23,mob,None,202312312352_1,202312312352_14
35,EFMTX;6591ee0801021b00010facb4,adsl,adsl,2023-12-31 23:41:12,2023-12-31 23:57:48,2023-12-31 23:58:02,sassitbn/adsl/100,sass,2023-12-31 23:39:08,tx,None,202312312342_13,202312312340_13
37,EFMTX;6591efb9d7d6bc00019444d6,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:57:14,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13
38,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13


In [12]:
clusters_filtered_grouped = clusters_filtered.groupby(["cluster_id2"])

In [13]:
clusters_size = clusters_filtered_grouped.size()

In [14]:
clusters_size.sort_values(ascending=False)

cluster_id2
202309181126_19    10079
202308080334_40     7989
202311270204_5      6728
202311270406_7      6611
202310110506_13     6226
                   ...  
202303240120_2         3
202303240122_4         3
202303240124_9         3
202309041024_13        3
202307251122_54        3
Length: 930871, dtype: int64

In [15]:
# Supponiamo che tu abbia un DataFrame raggruppato groupby_df e che tu stia cercando gruppi con almeno 2 'ne_id' unici
clusters_filtered = clusters.filter(lambda x: x['ne_id'].nunique() >= 2)

In [16]:
clusters_filtered

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
16,COS1P;136885076,sdh,ad16,2022-12-31 03:18:22,2022-12-31 23:59:59,2023-01-01 00:00:02,bracitah/ad16/1,brac,2022-12-31 03:18:22,tx,None,202301010000_31,202301010000_16
17,COS1P;136885115,sdh,ad16,2022-12-31 03:18:26,2022-12-31 23:59:50,2023-01-01 00:00:02,bracitam/ad16/3,brac,2022-12-31 03:18:26,tx,None,202301010000_32,202301010000_16
26,COS1P;136884642,sdh,adm4,2022-12-31 03:15:35,2022-12-31 23:59:52,2023-01-01 00:00:02,capoitam/adm4/1,capo,2022-12-31 03:15:35,tx,None,202301010000_37,202301010000_25
27,COS1P;136884882,sdh,ad16,2022-12-31 03:17:37,2022-12-31 23:59:56,2023-01-01 00:00:02,capoitaq/ad16/1,capo,2022-12-31 03:17:37,tx,None,202301010000_38,202301010000_25
28,COS1P;136884883,sdh,ad16,2022-12-31 03:17:37,2022-12-31 23:59:50,2023-01-01 00:00:02,capoitaq/ad16/1,capo,2022-12-31 03:17:37,tx,None,202301010000_38,202301010000_25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,EFM;6591f07dadefac0001744e96,an,vnbts,2023-12-31 23:51:41,2023-12-31 23:56:28,2023-12-31 23:58:03,7btsre15f,reem,2023-12-31 23:51:41,mob,None,202312312358_12,202312312354_7
32,EFM;6591f08739628d00013784db,an,vnbts,2023-12-31 23:51:51,2023-12-31 23:56:44,2023-12-31 23:58:03,7btsre0b9,reem,2023-12-31 23:51:51,mob,None,202312312358_11,202312312354_7
33,EFM;6591f08cadefac0001744ea1,an,vnbts,2023-12-31 23:51:56,2023-12-31 23:56:28,2023-12-31 23:58:03,7btsre04b,reem,2023-12-31 23:51:56,mob,None,202312312358_10,202312312354_7
35,EFMTX;6591ee0801021b00010facb4,adsl,adsl,2023-12-31 23:41:12,2023-12-31 23:57:48,2023-12-31 23:58:02,sassitbn/adsl/100,sass,2023-12-31 23:39:08,tx,None,202312312342_13,202312312340_13


In [28]:
clusters_filtered.to_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [17]:
grouped_clusters = clusters_filtered.groupby(["cluster_id2"])

In [27]:
len(grouped_clusters)

864160

In [25]:
# Ottieni il primo gruppo
first_group_key = list(grouped_clusters.groups.keys())[300]
first_group = grouped_clusters.get_group(first_group_key)

first_group

C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_14424\187166136.py:3: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  first_group = grouped_clusters.get_group(first_group_key)


,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
6,EFM;63afe4a9d7aa710001993702,an,lebts,2022-12-31 08:28:41,2023-01-01 06:24:02,2023-01-01 06:26:03,8btslt008,form,2022-12-31 08:28:41,mob,None,202301010626_2,202301010626_6
10,EFM;63afe39bd7aa710001993700,an,lebts,2022-12-31 08:24:11,2023-01-01 06:41:02,2023-01-01 06:42:03,2btslt008,form,2022-12-31 08:24:11,mob,None,202301010642_1,202301010626_6


In [26]:
first_group.sort_values(by=["first_occurrence"])

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
10,EFM;63afe39bd7aa710001993700,an,lebts,2022-12-31 08:24:11,2023-01-01 06:41:02,2023-01-01 06:42:03,2btslt008,form,2022-12-31 08:24:11,mob,None,202301010642_1,202301010626_6
6,EFM;63afe4a9d7aa710001993702,an,lebts,2022-12-31 08:28:41,2023-01-01 06:24:02,2023-01-01 06:26:03,8btslt008,form,2022-12-31 08:28:41,mob,None,202301010626_2,202301010626_6
